# JUAS Longitudinal tutorial 4: acceleration with phase change at transition

This notebooks shows a full transition crossing example, and generates a video of the longitudinal phase space.

It is based on an xtrack script: https://github.com/xsuite/xtrack/blob/main/examples/transition_crossing/002_transition_crossing.py

In [ ]:
# Import basic libraries
import numpy as np
from scipy.constants import m_p, c, e
from scipy.interpolate import interp1d

# Import from Xsuite
import xtrack as xt
import xpart as xp

# Matplotlib and animation
import matplotlib.pyplot as plt
import matplotlib.animation as ani

# Matplotlib default parameters for figure size and font size
plt.rcParams.update({'font.size': 16})
plt.rcParams.update({'figure.figsize': (8, 6)})
plt.rcParams.update({'figure.autolayout': True})

# LaTeX printing
from IPython.display import display, Markdown

import warnings
warnings.filterwarnings('ignore')

# We will use FFMpeg for saving animations
# This requires `conda install -c conda-forge ffmpeg`
from matplotlib.animation import FFMpegFileWriter

In [ ]:
####################################
# INPUT HERE Simulation parameters #
####################################

# Machine parameters: circumference and Twiss parameters at injection point
circumference = 100 * 2 * np.pi # in [m]

inj_alpha_x = 0
inj_alpha_y = 0
inj_beta_x = 16.
inj_beta_y = 16.

# Machine parameters: tunes, gamma transition, momentum compaction factor
Qx = 6.3
Qy = 6.4
gamma_tr = 6.1
alpha_c = gamma_tr**(-2)

# Machine parameters: RF voltage, harmonic number, acceleration gradient, bending radius and phase offset
V_rf = 200e3 # in [V]
harmonic= 8

Bdot =  2.2         # in T/s
bending_radius = 70 # in [m]

# Beam parameters: initial kinetic energy, total intensity, transverse emittances
Ekin = 1.4e9       # in [eV]
intensity = 8e12
epsn_x = 5e-6      # in [m*rad]
epsn_y = 5e-6      # in [m*rad]

# Calculations: Lorentz gamma and beta
gamma = 1 + e * Ekin / (m_p * c**2)
beta = np.sqrt(1 - gamma**-2)


# Beam parameter: initial bunch length
bunch_length = 2.5*230e-9/8 # 20e-9  # full bunch length in [s]
sigma_z_0 = (bunch_length / 4) * beta * c  # rms bunch length in [m]

display(Markdown(rf'Initial bunch length: $\sigma_{{z, 0}}={sigma_z_0:.4G}$ m'))

# Number of sections in which the accelerator circumference is divided (keep it at 1)
n_segments = 1


####################################
# END OF INPUT                     #
####################################

# Calculations: total energy
Etot = gamma * m_p * c**2 / e
display(Markdown(rf'$\beta_{{Lorentz}} = {beta:.4f}$   $\gamma_{{Lorentz}} = {gamma:.4f}$ $E_{{total}} = {Etot/1e9:.3G}$ GeV'))

# Calculations: slippage factor and phase offset
eta = alpha_c - gamma**-2
display(Markdown(rf'$\eta = {eta:.4G}$'))

# Calculation: initial momentum and synchrotron rune
p0 = np.sqrt(gamma**2 - 1) * m_p * c
Qs = np.sqrt(np.abs(eta) * V_rf * harmonic/ (2 * np.pi * beta**2 * Etot)) # linear synchrotron tune without acceleration
display(Markdown(rf'$Q_s = {Qs}$'))

# Calculation: revolution time, energy increment per turn
turn_period = circumference / (beta * c)
energy_increment_per_turn = e * circumference * bending_radius * Bdot  # in [J]
energy_increment_per_turn_eV = energy_increment_per_turn / e  # in [eV]

display(Markdown(rf'Revolution time: $T_0 = {turn_period*1e6:.4G}$ $\mu s$'))
display(Markdown(rf'Energy increment per turn: $\Delta E_{{turn}} = {energy_increment_per_turn_eV/1e3:.4G}$ keV'))

#
compensate_phase = True
if compensate_phase:
    phi_below = np.arcsin(bending_radius*circumference*Bdot/V_rf)
    phi_above = np.pi - phi_below
    display(Markdown(rf'RF phase (rad) below transition: $\phi_{{s, below}} = {phi_below:.4G}$ rad'))
    display(Markdown(rf'RF phase (rad) above transition: $\phi_{{s, above}} = {phi_above:.4G}$ rad'))
    
    lag_rf_above = np.rad2deg(phi_above)
    lag_rf_below = np.rad2deg(phi_below)
    display(Markdown(rf'RF phase below transition: $\phi_{{s, below}} = {lag_rf_below:.4G}^o$'))
    display(Markdown(rf'RF phase above transition: $\phi_{{s, above}} = {lag_rf_above:.4G}^o$'))
    
    if eta > 0:
        lag_rf = lag_rf_above
    else:
        lag_rf = lag_rf_below

### Tracking parameters

In [ ]:
num_particles_to_track = 5_000
num_turns_to_track = 50_000

## Accelerator map

In [ ]:
# Create the one turn transfer matrix
matrix = xt.LineSegmentMap(
            qx=Qx, qy=Qy,
            dqx=0, dqy=0,
            betx=inj_beta_x, alfx=inj_alpha_x,
            bety=inj_beta_y, alfy=inj_alpha_y,
            dx=0, dpx=0,
            dy=0, dpy=0,
            voltage_rf=V_rf,
            frequency_rf=harmonic*1/turn_period,
            lag_rf=lag_rf,  # in degrees for XSuite
            momentum_compaction_factor=alpha_c,
            longitudinal_mode="nonlinear",
            length=circumference,
            energy_ref_increment=energy_increment_per_turn_eV,
            energy_increment=0,
            )

# Create the line
line = xt.Line(elements={'one_turn_map': matrix,})
line.particle_ref = xt.Particles(mass0=xp.PROTON_MASS_EV, gamma0=gamma, q0=1.0)

In [ ]:
tw = line.twiss()

p, matcher = xp.generate_matched_gaussian_bunch(
    line=line,
    num_particles=num_particles_to_track,
    nemitt_x=epsn_x,
    nemitt_y=epsn_y,
    sigma_z=sigma_z_0,
    return_matcher=True)


# Logger (log every ten turns)
log_every = 500
n_log = num_turns_to_track // log_every
mon = xt.ParticlesMonitor(
    start_at_turn=0,
    stop_at_turn=1,
    n_repetitions=n_log,
    repetition_period=log_every,
    num_particles=len(p.x))

In [ ]:
#########
# Track #
#########

jumped = False
i_jumped = None
while p.at_turn[0] < num_turns_to_track:
    print(f'Turn {p.at_turn[0]}/{num_turns_to_track}            ', end='\r', flush=True)

    # Phase jump
    if p.gamma0[0] > gamma_tr and not jumped:
        print(f'Jumped at turn: {p.at_turn[0]}')
        line['one_turn_map'].lag_rf = lag_rf_above
        i_jumped = p.at_turn[0]
        jumped = True

    # Track
    line.track(p, num_turns=100, turn_by_turn_monitor=mon)
    p.reorganize() # (put lost particles at the end)


In [ ]:
############
# Plotting #
############

# Generate separatrix for video
i_separatrix = []
z_separatrix = []
delta_separatrix = []
# for i_sep in np.arange(0, num_turns_to_track, 100):
for i_sep in np.arange(0, num_turns_to_track, log_every):
    print(f'Separatrix {i_sep}/{num_turns_to_track}            ', end='\r', flush=True)
    line.particle_ref.gamma0 = mon.gamma0[i_sep//log_every, 0, 0]
    line['one_turn_map'].lag_rf = lag_rf_above if i_sep >= i_jumped else lag_rf_below
    try:
        rfb = line._get_bucket()
        i_separatrix.append(i_sep)
        z_separatrix.append(np.linspace(rfb.z_left, rfb.z_right, 1000))
        delta_separatrix.append(rfb.separatrix(z_separatrix[-1]))
    except:
        i_separatrix.append(i_sep)
        z_separatrix.append(np.zeros(1000))
        delta_separatrix.append(np.zeros(1000))
        z_separatrix[-1][:] = -1e20
        delta_separatrix[-1][:] = 0

sigma_z_rms = np.squeeze(mon.zeta.std(axis=1))
z_separatrix = np.array(z_separatrix)
delta_separatrix = np.array(delta_separatrix)
i_separatrix = np.array(i_separatrix)

plt.close('all')
fig1 = plt.figure(1)
plt.plot(mon.at_turn[:, 0, 0], sigma_z_rms)
plt.xlabel('Turn')
plt.ylabel('Bunch length [m]')
plt.show()


f_sep_z = interp1d(i_separatrix, z_separatrix, axis=0,
                   bounds_error=False, fill_value='extrapolate')
f_sep_delta = interp1d(i_separatrix, delta_separatrix, axis=0,
                       bounds_error=False, fill_value='extrapolate')

In [ ]:
np.rad2deg(phi_above)

In [ ]:
# Make movie (needed `conda install -c conda-forge ffmpeg``)
def update_plot(i_log, fig):
    i_turn = mon.at_turn[i_log, 0, 0]
    z_sep = f_sep_z(i_turn)
    delta_sep = f_sep_delta(i_turn)

    plot_separatrix = True
    if np.abs(i_turn - i_jumped) < 100:
        plot_separatrix = False

    phi_rf_deg = np.rad2deg(phi_above) if i_turn >= i_jumped else np.rad2deg(phi_below)
    plt.clf()
    plt.plot(mon.zeta[i_log, :], mon.delta[i_log, :], '.', markersize=1)
    if plot_separatrix:
        plt.plot(z_sep, delta_sep, color='C1', linewidth=2)
        plt.plot(z_sep, -delta_sep, color='C1', linewidth=2)
    plt.xlim(-40, 40)
    plt.ylim(-20e-3, 20e-3)
    plt.xlabel('z [m]')
    plt.ylabel(r'$\Delta p / p_0$')
    plt.title(f'Turn {i_turn} '
              r'$\sigma_\zeta = $' f'{sigma_z_rms[i_log]:.2f}\n'
              r'$\gamma_0 = $' f'{mon.gamma0[i_log, 0, 0]:.2f} '
              r'$\gamma_t = $' f'{gamma_tr:.2f} '
              r'$\phi_{\mathrm{rf}} = $' f'{phi_rf_deg:.2f}'
              )
    plt.subplots_adjust(left=0.2, top=0.82)
    plt.grid(alpha=0.5)

fig = plt.figure()

moviewriter = FFMpegFileWriter(fps=15)
with moviewriter.saving(fig, 'transition_crossing_PS.mp4', dpi=100):
    for j in range(0, len(mon.zeta[:, 0, 0]), 1):
        print(f'Frame {j}/{len(mon.zeta[:, 0, 0])}            ', end='\r', flush=True)
        update_plot(j, fig)
        moviewriter.grab_frame()